<a href="https://colab.research.google.com/github/WeustiS/ProjectX/blob/main/Data_Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as r
from io import BytesIO, StringIO
import zipfile
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from google.colab import drive
import time
import pickle

In [ ]:
base_url = r'http://oasis.caiso.com/oasisapi/SingleZip?'
# ?queryname=PRC_AS&startdatetime=20191018T07:00-0000&enddatetime=20191019T07:00-0000
# &market_run_id=DAM&version=1&as_type=ALL&as_region=ALL

def get_data(params):
    # building URL
    url = base_url
    for key, value in params.items():
        url += f"{key}={value}&"
    data = r.get(url[:-1]).content
    print("URL:", url)
    try:
        filecontent = BytesIO(data)
    except TypeError:
        filecontent = StringIO(data)

    try:
            # have zipfile
        z = zipfile.ZipFile(filecontent)
    except zipfile.BadZipfile:
        return None

        # have unzipped content
    unzipped = [z.read(thisfile) for thisfile in z.namelist()]
    z.close()

        # return
    return unzipped

In [ ]:
# function to convert XML to 
def parse_XML(root):
  df = pd.DataFrame(columns=['Variable', 'Node', 'Date','Interval Number','Value'])
  j = 0
  #
  for i in root[1][0][4][1:]:
    #print(i[2].text, float(i[6].text))\
    new_row = {'Variable':i[0].text, 'Node':i[1].text, 'Date':i[2].text, 'Interval Number':float(i[3].text),'Value':float(i[6].text)}
    #append row to the dataframe
    df = df.append(new_row, ignore_index=True)
  return df

In [ ]:
# function to convert XML to 
def parse_XML_energy(root):
  df = pd.DataFrame(columns=['Variable', 'Date','Interval Number','Value', 'Trading Hub', 'Renewable Energy Type'])
  j = 0
  #
  for i in root[1][0][4][1:]:
    #print(i[2].text, float(i[6].text))\
    new_row = {'Variable':i[0].text, 'Node':i[1].text, 'Date':i[2].text, 'Interval Number':float(i[3].text),'Value':float(i[6].text)}
    #append row to the dataframe
    df = df.append(new_row, ignore_index=True)
  return df

In [ ]:
#trial cell
#payload = {
#            'queryname': 'PRC_INTVL_LMP', 
#            'startdatetime': '20200102T08:00-0000',
#            'enddatetime':'20200103T08:00-0000',
#            'version': '1',
#            'market_run_id': 'RTM',
#            'node': 'PERRY_6_N001'
#          }
##xml = get_data(payload)
#xml_to_string = str(xml[0],'utf-8')
#display(xml_to_string)
#root = ET.fromstring(xml_to_string)
#data = parse_XML(root)
##DF_list.append(data)


In [ ]:
# # function to convert XML to 
# def parse_XML_energy(root):
#   df = pd.DataFrame(columns=['Variable', 'Date','Interval Number','Value', 'Trading Hub', 'Renewable Energy Type'])
#   for j in range(1,7):
#     for i in root[1][0][j][1:]:
#       print(i[2].text, float(i[5].text))
#       #new_row = {'Variable':i[0].text, 'Date':i[1].text, 'Interval Number':int(i[2].text), 'Value':float(i[5].text),'Trading Hub':i[6].text, 'Renewable Energy Type':i[7].text}
#     #append row to the dataframe0
#     #df = df.append(new_row, ignore_index=True)
#  # return df

In [ ]:
# function to convert XML to 
def parse_XML_energy(root):
  df = pd.DataFrame(columns=['Variable', 'Date','Interval Number','Value', 'Trading Hub', 'Renewable Energy Type'])
  for j in range(1,7):
    for i in root[1][0][j][1:]:
      #print(i[2].text, float(i[6].text))\
      new_row = {'Variable':i[0].text, 'Date':i[1].text, 'Interval Number':i[2].text, 'Value':float(i[5].text),'Trading Hub':i[6].text, 'Renewable Energy Type':i[7].text}
      df = df.append(new_row, ignore_index=True)
    #append row to the dataframe0
  return df

In [ ]:
DF_list= list()
payload = {
           'queryname': 'SLD_REN_FCST', 
           'startdatetime': '20200102T08:00-0000',
           'enddatetime':'20200103T08:00-0000',
           'version': '1',
           'market_run_id': 'RTD',
           #'node': 'PERRY_6_N001'
         }
xml = get_data(payload)
xml_to_string = str(xml[0],'utf-8')
display(xml_to_string)
root = ET.fromstring(xml_to_string)
data = parse_XML_energy(root)
DF_list.append(data)

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20200102T08:00-0000&enddatetime=20200103T08:00-0000&version=1&market_run_id=RTD&


'<?xml version="1.0" encoding="UTF-8"?><OASISReport xmlns="http://www.caiso.com/soa/OASISReport_v1.xsd">\n<MessageHeader>\n<TimeDate>2020-11-24T20:56:37-00:00</TimeDate>\n<Source>OASIS</Source>\n<Version>v20140401</Version>\n</MessageHeader>\n<MessagePayload>\n<RTO>\n<name>CAISO</name>\n<REPORT_ITEM>\n<REPORT_HEADER>\n<SYSTEM>OASIS</SYSTEM>\n<TZ>PPT</TZ>\n<REPORT>SLD_REN_FCST</REPORT>\n<MKT_TYPE>RTD</MKT_TYPE>\n<UOM>MW</UOM>\n<INTERVAL>ENDING</INTERVAL>\n<SEC_PER_INTERVAL>300</SEC_PER_INTERVAL>\n</REPORT_HEADER>\n<REPORT_DATA>\n<DATA_ITEM>RENEW_FCST_5MIN_MW</DATA_ITEM>\n<OPR_DATE>2020-01-02</OPR_DATE>\n<INTERVAL_NUM>1</INTERVAL_NUM>\n<INTERVAL_START_GMT>2020-01-02T08:00:00-00:00</INTERVAL_START_GMT>\n<INTERVAL_END_GMT>2020-01-02T08:05:00-00:00</INTERVAL_END_GMT>\n<VALUE>0</VALUE>\n<TRADING_HUB>NP15</TRADING_HUB>\n<RENEWABLE_TYPE>Solar</RENEWABLE_TYPE>\n</REPORT_DATA>\n<REPORT_DATA>\n<DATA_ITEM>RENEW_FCST_5MIN_MW</DATA_ITEM>\n<OPR_DATE>2020-01-02</OPR_DATE>\n<INTERVAL_NUM>2</INTERVAL_NU

In [ ]:
market_type = {'RTD', 'ACTUAL','DAM','HASP','RTPD'}

In [ ]:
nodes_list_1 = ['WOHLER_6_N001', 'SOLEDAD_6_N001', 'ZAMORA_1_N001', 'SILVERDO_1_N002', 'ZANKER_1_N003']
nodes_list_2 = ['WINTERS_6_N001', 'REDBUD_1_N004']
nodes_list_3 = ['PERRY_6_N001', 'SANDBAR_7_B1', 'TULUCAY_6_N001']
nodes_list_4 = ['WOODSIDE_6_N004', 'WOODACRE_6_N001', 'ROBROY_1_N001', 'WATRSHED_6_N001','OLEMA_6_N001']
nodes_list_5 = ['BOLINAS_6_N001', 'TOCALOMA_6_N001','STONYPT_1_N001','STAFFORD_6_N101','WOLFSKIL_7_B1']
nodes_list_6 = ['VICTOR_6_N002', 'SPRINGVL_6_N006', 'PHILO_6_N001','MALCHA_7_B1', 'EASTWOOD_7_B1', 'CONTROLX_1_N010', 'WAHTOKE_1_N002', 'RTRACK_1_N001', 'QUEBEC_1_N001', 'WHEATLND_6_N006']
nodes_list_8 = ['MUSLSLGH_6_N001', 'S0479_7_N003', 'WEMRSWS_6_N001', 'SHNGLESR_1_N103','JOLON_6_N001']
nodes_list_11 = ['STMARIA_1_N101', 'OCEANO_1_N004','GRANDIS_1_N009', 'INDIANV_7_B1','WHEELBR1_7_B1']


In [ ]:
DF_list= list()
for market in market_type:
  time.sleep(2)
  for month in range(11, 13):
    time.sleep(2)
    for i in range(1,31):
      if (month == 2 and i > 27):
        continue
      if ((month == 4 or month == 6 or month == 9 or month == 11) and i > 29):
        continue
      time.sleep(5)
      if (month < 10):
        if (i < 10):
          # changing the start and end date for scraping purposes
          start_date = '20190' + str(month)+'0' + str(i)+'T08:00-0000'
          end_date = '20190'+str(month) + '0'+str(i+1)+'T08:00-0000'
          # dynamically changing payload
          payload = {
            'queryname': 'SLD_REN_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            #'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          # converting xml to dataframe
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_energy(root)
          DF_list.append(data)
        else:
          start_date = '20190' + str(month)+ str(i)+'T08:00-0000'
          end_date = '20190'+str(month) + str(i+1)+'T08:00-0000'
          payload = {
            'queryname': 'SLD_REN_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            #'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_energy(root)
          DF_list.append(data)
      else:
        if (i < 10):
          # changing the start and end date for scraping purposes
          start_date = '2019' + str(month)+ '0'+ str(i)+'T08:00-0000'
          end_date = '2019'+str(month)+'0'+str(i+1)+'T08:00-0000'
          # dynamically changing payload
          payload = {
            'queryname': 'SLD_REN_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
          'market_run_id': market,
          #'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          # converting xml to dataframe
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_energy(root)
          DF_list.append(data)
        else:
          start_date = '2019' + str(month)+ str(i)+'T08:00-0000'
          end_date = '2019' + str(month)+str(i+1)+'T08:00-0000'
          payload = {
            'queryname': 'SLD_REN_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            #'node': node
          }
          xml = get_data(payload)
          if (xml is None):
            continue
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_energy(root)
          DF_list.append(data)

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20191101T08:00-0000&enddatetime=20191102T08:00-0000&version=1&market_run_id=ACTUAL&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20191102T08:00-0000&enddatetime=20191103T08:00-0000&version=1&market_run_id=ACTUAL&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20191103T08:00-0000&enddatetime=20191104T08:00-0000&version=1&market_run_id=ACTUAL&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20191104T08:00-0000&enddatetime=20191105T08:00-0000&version=1&market_run_id=ACTUAL&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20191105T08:00-0000&enddatetime=20191106T08:00-0000&version=1&market_run_id=ACTUAL&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20191106T08:00-0000&enddatetime=20191107T08:00-0000&version=1&market_run_id=

In [ ]:
DF_list
df = pd.concat(DF_list)
df

,Variable,Date,Interval Number,Value,Trading Hub,Renewable Energy Type
0,RENEW_FCST_ACT_MW,2019-11-01,2,-3.52425,NP15,Solar
1,RENEW_FCST_ACT_MW,2019-11-01,3,-3.54080,NP15,Solar
2,RENEW_FCST_ACT_MW,2019-11-01,4,-3.55326,NP15,Solar
3,RENEW_FCST_ACT_MW,2019-11-01,5,-3.56082,NP15,Solar
4,RENEW_FCST_ACT_MW,2019-11-01,6,-3.52754,NP15,Solar
...,...,...,...,...,...,...
115,RENEW_FCST_HASP_MW,2019-12-30,20,0.00000,ZP26,Solar
116,RENEW_FCST_HASP_MW,2019-12-30,21,0.00000,ZP26,Solar
117,RENEW_FCST_HASP_MW,2019-12-30,22,0.00000,ZP26,Solar
118,RENEW_FCST_HASP_MW,2019-12-30,23,0.00000,ZP26,Solar


In [ ]:
pickle.dump(df, open("2019_energy_nov_dec", "wb"))

In [ ]:
# code to save data in google drive directly
#data = parse_XML(root)
drive.mount('drive', force_remount=True)
df.to_csv('2019_energy_nov_dec.csv', index = False)
!cp 2019_energy_nov_dec.csv "drive/My Drive/ProjectX Data"

Mounted at drive


In [ ]:
00

0